In [46]:
!pip install train_mnist

ERROR: Could not find a version that satisfies the requirement train_mnist (from versions: none)
ERROR: No matching distribution found for train_mnist


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim

class MNIST(nn.Module):
    def __init__(self):
        super(MNIST, self).__init__()
        self.conv0  = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, bias=False) #1->20: 20個特徵提取器；長寬28x28->24x24
        self.bn0    = nn.BatchNorm2d(num_features=20) #依該批量data之平均及標準差normalize到1~-1之間，再乘上更新權重Gamma + Beta，並紀錄經驗供預測時用 running_mean/var
        self.relu0  = nn.ReLU()
        self.maxp0  = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) #->長寬24x24 -> 12x12

        self.conv1  = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, bias=False) #長寬12x12->8x8
        self.bn1    = nn.BatchNorm2d(num_features=50)
        self.relu1  = nn.ReLU()
        self.maxp1  = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))#長寬8x8->4x4   1x50x4x4

        self.conv2  = nn.Conv2d(in_channels=50, out_channels=500, kernel_size=4, stride=1, bias=False) #長寬4x4 -> 1x1
        self.bn2    = nn.BatchNorm2d(num_features=500)
        self.relu2  = nn.ReLU()
        self.conv3  = nn.Conv2d(in_channels=500, out_channels=10, kernel_size=1, stride=1, bias=False) #長寬1x1->1x1 ;特徵500->10
        # self.dense2 = nn.Linear(in_features=400, out_features=120, bias=False) #2x400X400x120 -> 2x120
        # self.dp2    = nn.Dropout(p=0.5)
        # self.relu2  = nn.ReLU()
        # self.dense3 = nn.Linear(in_features=120, out_features=10, bias=False)

    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.maxp0(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.maxp1(x)

        # x = x.view(-1, self.num_flat_features(x))
        # x = self.dense2(x)
        # x = self.dp2(x)
        # x = self.relu2(x)
        # x = self.dense3(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = torch.squeeze(x) #1x10x1x1 ->1x10
        return x

In [47]:
# from train_mnist import MNIST
model = MNIST()

# Load the weights from a file (.pth usually)
weights_path = './mnist.pth'
state_dict = torch.load(weights_path)

# Load the weights now into a model net architecture defined by our class
model.load_state_dict(state_dict)

input = torch.randn(1, 1, 28, 28)

torch.onnx.export(model, input, "mnist.onnx", verbose=True)

import onnx_chainer
model = onnx.load("mnist.onnx")

onnx.checker.check_model(model)

onnx.helper.printable_graph(model.graph)

FileNotFoundError: ignored